<a href="https://colab.research.google.com/github/MahayaImad/PhD-Thesis-Privacy-AI/blob/main/Training_experiments_with_grid_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Parameters

In [ ]:
# "convnext" "densenet" "efficientnet" "efficientnet_v2""inception_resnet_v2""inception_v3""mobilenet_v3" "resnet50" "resnet_v2" "vgg16" "vgg19" "xception"
MODEL_ARCHITECTURE = "mobilenet_v3"
EPOCHS = 10
TRANFERT_LEARNING =True
#==============================#

DATASET = "pathmnist_128"
INPUT_SHAPE = (128, 128, 3)
CLASSES = 9
#==============================#
param_grid = {
    "learning_rate": [1e-4, 1e-3],
    "batch_size": [32, 64, 128],
    "optimizer": ["adam", "sgd"],
    "dropout": [0.2, 0.3, 0.4, 0.5],
    "dense_units": [64, 128, 256]
}
# param_grid = {
#     "learning_rate": [1e-4],
#     "batch_size": [64],
#     "optimizer": ["adam", "sgd"],
#     "dropout": [0.2],
#     "dense_units": [64]
# }
#==============================#
base_path = '/content/drive/MyDrive/ColabNotebooks/phd-thesis'

## Importing Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import sys
import tensorflow
import pandas as pd

from sklearn.model_selection import ParameterGrid
from tensorflow.keras.models import load_model
sys.path.append('/content/drive/MyDrive/ColabNotebooks/phd-thesis/lib')

#importing helpers
from import_pathmnist_dataset import PathMNISTLoader
from create_model import create_model
from import_callbacks import get_callbacks
from helper_saving import save_training

# To save in Excel file
# hyperparams = params

from datetime import datetime
experiment_info = {
    "model" : MODEL_ARCHITECTURE,
    "dataset" : DATASET,
    "mode" : "Local",
    "run_time" : datetime.now().strftime("%Y-%m-%d %H:%M:%S")
}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import Dataset

In [ ]:
loader = PathMNISTLoader(filename= DATASET, num_classes= CLASSES, batch_size= 64)
train_ds, val_ds, test_ds = loader.get_datasets()

Download complete.
Loaded data keys: ['train_images', 'train_labels', 'val_images', 'val_labels', 'test_images', 'test_labels']
Train: (89996, 128, 128, 3) (89996, 1)
Validation: (10004, 128, 128, 3) (10004, 1)
Test: (7180, 128, 128, 3) (7180, 1)


## Train and Save Results

In [ ]:
# Initialize an empty list to collect results
results = []

for i,config in enumerate(ParameterGrid(param_grid)):
  model = create_model(
    architecture=MODEL_ARCHITECTURE,
    input_shape=INPUT_SHAPE,
    num_classes=CLASSES,
    dropout_rate=config['dropout'],
    learning_rate=config['learning_rate'],
    optimizer=config['optimizer'],
    dense_units=config['dense_units'],
    freeze_base=TRANFERT_LEARNING
    )
  # Get callbacks
  callbacks = get_callbacks(
      model_name = f'{MODEL_ARCHITECTURE}_{DATASET}_{datetime.now().strftime("%Y-%m-%d")}_{i}',
      checkpoint_dir = os.path.join(base_path, MODEL_ARCHITECTURE,'models'),
      )

  # Train the model
  history = model.fit(
      train_ds,
      validation_data=val_ds,
      epochs=20,
      callbacks=callbacks,
      verbose=2,
  )

  SAVE_PATH=os.path.join(base_path, MODEL_ARCHITECTURE,'results', f'{MODEL_ARCHITECTURE}_{DATASET}_{datetime.now().strftime("%Y-%m-%d")}_{i}.xlsx')
  save_training(save_path=SAVE_PATH,model=model,history = history,hyperparams=config,experiment_info=experiment_info)

  model = load_model(os.path.join(base_path, MODEL_ARCHITECTURE,'models',f'{MODEL_ARCHITECTURE}_{DATASET}_{datetime.now().strftime("%Y-%m-%d")}_{i}.keras'))
  # Evaluate on the test set
  test_loss, test_accuracy = model.evaluate(test_ds)

  # Get best validation accuracy & loss
  best_val_acc = max(history.history["val_accuracy"])
  best_val_loss = min(history.history["val_loss"])

  # Log all info into a result dictionary
  result_record = {
      "model": MODEL_ARCHITECTURE,
      "run_id": i,
      "test_accuracy": test_accuracy,
      "test_loss": test_loss,
      "best_val_accuracy": best_val_acc,
      "best_val_loss": best_val_loss,
      **config  # unpack all hyperparameters
  }
  results.append(result_record)

results_df = pd.DataFrame(results)
results_df.to_excel(os.path.join(base_path, MODEL_ARCHITECTURE,'results', f'{MODEL_ARCHITECTURE}_{DATASET}_{datetime.now().strftime("%Y-%m-%d")}.xlsx'), index=False)


/usr/local/lib/python3.11/dist-packages/keras/src/applications/mobilenet_v3.py:452: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


1407/1407 - 34s - 24ms/step - accuracy: 0.2674 - loss: 2.0316 - val_accuracy: 0.3447 - val_loss: 1.8574
Epoch 2/20
1407/1407 - 15s - 11ms/step - accuracy: 0.3297 - loss: 1.7907 - val_accuracy: 0.3768 - val_loss: 1.7043
Epoch 3/20
1407/1407 - 16s - 11ms/step - accuracy: 0.3491 - loss: 1.7019 - val_accuracy: 0.3948 - val_loss: 1.6432
Epoch 4/20
1407/1407 - 16s - 11ms/step - accuracy: 0.3653 - loss: 1.6576 - val_accuracy: 0.4004 - val_loss: 1.6082
Epoch 5/20
1407/1407 - 15s - 11ms/step - accuracy: 0.3773 - loss: 1.6302 - val_accuracy: 0.4065 - val_loss: 1.5856
Epoch 6/20
1407/1407 - 16s - 11ms/step - accuracy: 0.3875 - loss: 1.6062 - val_accuracy: 0.4113 - val_loss: 1.5675
Epoch 7/20
1407/1407 - 16s - 11ms/step - accuracy: 0.3962 - loss: 1.5906 - val_accuracy: 0.4178 - val_loss: 1.5538
Epoch 8/20
1407/1407 - 15s - 11ms/step - accuracy: 0.4015 - loss: 1.5768 - val_accuracy: 0.4193 - val_loss: 1.5416
Epoch 9/20
1407/1407 - 15s - 11ms/step - accuracy: 0.4086 - loss: 1.5615 - val_accuracy: 0.

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()
/usr/local/lib/python3.11/dist-packages/keras/src/applications/mobilenet_v3.py:452: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


...

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()
/usr/local/lib/python3.11/dist-packages/keras/src/applications/mobilenet_v3.py:452: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


Epoch 1/20
1407/1407 - 33s - 23ms/step - accuracy: 0.1549 - loss: 2.1922 - val_accuracy: 0.2220 - val_loss: 2.1537
Epoch 2/20
1407/1407 - 15s - 11ms/step - accuracy: 0.2140 - loss: 2.1442 - val_accuracy: 0.2488 - val_loss: 2.1231
Epoch 3/20
1407/1407 - 15s - 11ms/step - accuracy: 0.2263 - loss: 2.1156 - val_accuracy: 0.2485 - val_loss: 2.0918
Epoch 4/20
1407/1407 - 15s - 11ms/step - accuracy: 0.2356 - loss: 2.0864 - val_accuracy: 0.2503 - val_loss: 2.0580
Epoch 5/20
1407/1407 - 16s - 12ms/step - accuracy: 0.2442 - loss: 2.0563 - val_accuracy: 0.3508 - val_loss: 2.0231
Epoch 6/20
1407/1407 - 15s - 11ms/step - accuracy: 0.2530 - loss: 2.0259 - val_accuracy: 0.3473 - val_loss: 1.9891
Epoch 7/20
1407/1407 - 15s - 11ms/step - accuracy: 0.2603 - loss: 1.9990 - val_accuracy: 0.3413 - val_loss: 1.9596
Epoch 8/20
1407/1407 - 15s - 11ms/step - accuracy: 0.2696 - loss: 1.9759 - val_accuracy: 0.3369 - val_loss: 1.9338
Epoch 9/20
1407/1407 - 15s - 11ms/step - accuracy: 0.2760 - loss: 1.9557 - val_a

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()
/usr/local/lib/python3.11/dist-packages/keras/src/applications/mobilenet_v3.py:452: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


Epoch 1/20
1407/1407 - 42s - 30ms/step - accuracy: 0.2934 - loss: 1.9503 - val_accuracy: 0.3762 - val_loss: 1.7646
Epoch 2/20
1407/1407 - 16s - 11ms/step - accuracy: 0.3474 - loss: 1.7205 - val_accuracy: 0.3953 - val_loss: 1.6501
Epoch 3/20
1407/1407 - 16s - 11ms/step - accuracy: 0.3689 - loss: 1.6498 - val_accuracy: 0.4039 - val_loss: 1.6021
Epoch 4/20
1407/1407 - 16s - 11ms/step - accuracy: 0.3847 - loss: 1.6099 - val_accuracy: 0.4069 - val_loss: 1.5723
Epoch 5/20
1407/1407 - 16s - 11ms/step - accuracy: 0.3962 - loss: 1.5835 - val_accuracy: 0.4152 - val_loss: 1.5514
Epoch 6/20
1407/1407 - 16s - 11ms/step - accuracy: 0.4074 - loss: 1.5624 - val_accuracy: 0.4224 - val_loss: 1.5346
Epoch 7/20
1407/1407 - 16s - 11ms/step - accuracy: 0.4151 - loss: 1.5449 - val_accuracy: 0.4294 - val_loss: 1.5187
Epoch 8/20
1407/1407 - 16s - 11ms/step - accuracy: 0.4215 - loss: 1.5314 - val_accuracy: 0.4351 - val_loss: 1.5067
Epoch 9/20
1407/1407 - 16s - 11ms/step - accuracy: 0.4291 - loss: 1.5171 - val_a

ValueError: File not found: filepath=/content/drive/MyDrive/ColabNotebooks/phd-thesis/mobilenet_v3/models/mobilenet_v3_pathmnist_128_2025-04-16_16.keras. Please ensure the file is an accessible `.keras` zip file.